<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-snow-dots-dotsPositions.csv-file" data-toc-modified-id="Load-snow-dots-dotsPositions.csv-file-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load snow-dots dotsPositions.csv file</a></span></li><li><span><a href="#Write-data-to-file-with-dotsDB" data-toc-modified-id="Write-data-to-file-with-dotsDB-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Write data to file with dotsDB</a></span></li></ul></div>

The purpose of this notebook is to write the dots positions dumped by snow-dots to a dotsDB hdf5 file.

# Load snow-dots dotsPositions.csv file

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv("../../data/pilot11_dotsPositions.csv")

In [4]:
data.head()

,xpos,ypos,isActive,isCoherent,frameIdx,trialCount
0,0.201351,0.464706,1,1,1,1
1,0.322217,0.292705,0,0,1,1
2,0.204674,0.105424,0,0,1,1
3,0.548188,0.340998,1,1,1,1
4,0.362038,0.515125,0,0,1,1


In [5]:
def snowDots2DotsDBNormalizedFrames(df):
    """
    Casts a pandas.DataFrame corresponding to the .csv file outputted by some snow-dots programs into a list of 
    lists of frames, as dotsDB accepts them. The main mapping is the swapping of x and y.
    
    In dotsDB, x represents vertical position, y horizontal, and (0,0) is top left corner.
    In snow-dots, x represents horizontal position, y vertical, and (0,0) is top left corner.
    
    .. Note::
        All trials must contain the same number of frames
    
    :param df: dataframe with columns "xpos", "ypos", "isActive", "isCoherence", "frameIdx", "trialCount" 
    :type df: pandas.DataFrame
    """
    list_of_trials = []
    num_trials = np.max(df["trialCount"])
    for tr in range(num_trials):
        list_of_frames = []
        trial_data = df[df["trialCount"] == (tr+1)]
        
        num_frames = np.max(trial_data["frameIdx"])
        assert not np.isnan(num_frames), f'trial {tr+1}, num_frames is {num_frames}'
        
        if tr == 0:
            first_num_frames = num_frames
            
        assert num_frames == first_num_frames, f'trial {tr+1} has {num_frames} frames against {first_num_frames} for trial 1'
        
        for fr in range(num_frames):
            frame_data = trial_data[(trial_data["frameIdx"] == (fr+1)) & (trial_data["isActive"] == 1)]
            list_of_frames.append(np.array(frame_data[['ypos','xpos']]))  # here I swap xpos with ypos
        
        list_of_trials.append(list_of_frames)
    return list_of_trials

# Write data to file with dotsDB

In [6]:
import sys
import pprint
# pprint.pprint(sys.path)

In [7]:
sys.path.append('../modules/dots_db/dotsDB')

In [8]:
import dotsDB as dDB

In [9]:
db_filename = '../../data/pilot11.h5'

In [10]:
smaller_data = data[data["frameIdx"] <= 6]

In [11]:
ddb_data = snowDots2DotsDBNormalizedFrames(smaller_data)

In [12]:
type(ddb_data)

list

In [13]:
len(ddb_data)

478

In [14]:
type(ddb_data[0])

list

In [15]:
len(ddb_data[0])

6

In [16]:
type(ddb_data[0][0])

numpy.ndarray

In [17]:
ddb_data[0][0].shape

(61, 2)

In [18]:
ddb_data[0][1].shape

(61, 2)

In [19]:
smaller_data.head()

,xpos,ypos,isActive,isCoherent,frameIdx,trialCount
0,0.201351,0.464706,1,1,1,1
1,0.322217,0.292705,0,0,1,1
2,0.204674,0.105424,0,0,1,1
3,0.548188,0.340998,1,1,1,1
4,0.362038,0.515125,0,0,1,1


In [20]:
ddb_data[0][0][:2,]

array([[0.46470647, 0.20135065],
       [0.34099791, 0.54818825]])

In [24]:
parameters = dict(speed=2.1,density=90,coh_mean=50,coh_stdev=10,direction='right',num_frames=6,diameter=10)

In [22]:
n_trials=len(ddb_data)

In [26]:
stimulus = dDB.DotsStimulus(**parameters)

In [27]:
%%time
dDB.write_stimulus_to_file(stimulus, n_trials, db_filename, create_file=True, pre_generated_stimulus=ddb_data)

CPU times: user 20 s, sys: 382 ms, total: 20.4 s
Wall time: 9.69 s


In [28]:
dDB.inspect_db(db_filename)

{'intlv3_lftTrue_fr60_fs1.1_sp2.1_ds90_c50_cs10_dright_nf6_dm10_sc5.0_ppd55.4612_dts6_fw610': {'type': 'group',
  'attrs': [('coh_mean', 50),
   ('coh_stdev', 10),
   ('density', 90),
   ('diameter', 10),
   ('direction', 'right'),
   ('dot_size_in_pxs', 6),
   ('field_scale', 1.1),
   ('frame_rate', 60),
   ('frame_width_in_pxs', 610),
   ('interleaves', 3),
   ('limit_life_time', True),
   ('num_frames', 6),
   ('pixels_per_degree', 55.4612),
   ('speed', 2.1),
   ('stencil_radius_in_vis_angle', 5.0)]},
 'intlv3_lftTrue_fr60_fs1.1_sp2.1_ds90_c50_cs10_dright_nf6_dm10_sc5.0_ppd55.4612_dts6_fw610/px': {'type': 'dataset',
  'attrs': [],
  'shape': (478, 2232600)}}